In [12]:
# install lib
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [13]:
# patch dpo
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [14]:
# load model
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/zephyr-sft-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.4.1: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [15]:
# load dataset
from datasets import load_dataset

# Load
dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split="train_prefs")

# Select small subset
dataset = dataset.select(range(500))

# Format properly
def format_for_dpo(example):
    return {
        "prompt": example["prompt"],
        "chosen": example["chosen"],
        "rejected": example["rejected"],
    }

# Make sure to remove unwanted columns like 'messages'
dataset = dataset.map(format_for_dpo, remove_columns=dataset.column_names)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [16]:
# add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 64,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [17]:
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig
from unsloth import is_bfloat16_supported

dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=DPOConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_ratio=0.1,
        num_train_epochs=1,
        learning_rate=5e-6,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.0,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
        report_to="none",
    ),
    beta=0.1,
    train_dataset=dataset,
    tokenizer=tokenizer,
    max_length=1024,
    max_prompt_length=512,
)

dpo_trainer.train()

Extracting prompt in train dataset (num_proc=12):   0%|          | 0/500 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=12):   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=12):   0%|          | 0/500 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 62
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 167,772,160/4,000,000,000 (4.19% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
1,0.693100,0.000000,0.000000,0.000000,0.000000,-111.377914,-242.882843,-2.614927,-2.597368,0,0,0,0
2,0.693100,0.000000,0.000000,0.000000,0.000000,-198.665741,-269.776611,-2.909889,-2.829716,No Log,No Log,No Log,No Log
3,0.690100,0.006724,-0.000481,0.625000,0.007204,-264.058228,-249.050766,-2.687579,-2.868556,No Log,No Log,No Log,No Log
4,0.697500,-0.003719,0.004726,0.375000,-0.008445,-196.744049,-282.480835,-2.714153,-2.970438,No Log,No Log,No Log,No Log
5,0.690200,-0.024067,-0.031855,0.375000,0.007788,-383.941101,-414.864685,-2.445025,-2.560859,No Log,No Log,No Log,No Log
6,0.696300,-0.015727,-0.010010,0.500000,-0.005716,-255.727966,-243.437363,-2.713888,-2.952719,No Log,No Log,No Log,No Log
7,0.636000,0.008410,-0.112998,0.875000,0.121408,-583.797058,-490.744965,-3.039751,-2.980752,No Log,No Log,No Log,No Log
8,0.684700,-0.075354,-0.094847,0.750000,0.019494,-286.952942,-280.610107,-2.969377,-3.013196,No Log,No Log,No Log,No Log
9,0.682300,-0.064396,-0.087857,0.375000,0.023461,-303.696014,-199.591599,-2.964472,-3.127069,No Log,No Log,No Log,No Log
10,0.649800,-0.078710,-0.167783,1.000000,0.089073,-355.231201,-401.109741,-2.867611,-2.975022,No Log,No Log,No Log,No Log


TrainOutput(global_step=62, training_loss=0.5624301385494971, metrics={'train_runtime': 277.7324, 'train_samples_per_second': 1.8, 'train_steps_per_second': 0.223, 'total_flos': 0.0, 'train_loss': 0.5624301385494971, 'epoch': 0.992})

In [18]:
# save LoRA
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')